# Wrangling


In [6]:
import pandas as pd

In [2]:
d = pd.read_csv("/Users/NikiAthanasiadou/FoR/Info-Data/Salaries/Postdoc_salary_data_Nov17.csv")
#d.fillna(value='None',inplace=True)
d.head()

,Institution,Title,Annual Salary,APPT FTR BASIS,APPT FRACTION,AMOUNT OF SALARY PAID FROM GENERAL FUND,%,FTE,Time,PAY_FRCTN,Last Name,First Name,Department,FT/PT,Contract Length,Degree,Highest Education Level,Type,Effective Date,Hire Date
0,ALBANY,Postdoctoral Associate,"$35,000.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALBANY,Postdoctoral Associate,"$35,901.32",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ALBANY,Postdoctoral Associate,"$36,000.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ALBANY,Postdoctoral Associate,"$36,000.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ALBANY,Postdoctoral Associate,"$36,000.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 1. Ethnicity and gender

In [3]:
#Create the two new columns (sex and ethnicity) and fill them with True/False depending on the presence of last/first name
d["Ethnea"] = d["Last Name"].notnull()
d["Genni"] = d["First Name"].notnull()

In [4]:
from urllib.request import urlopen
import ast

In [5]:
#Set up querry
for i in range(len(d)):
    url = "http://abel.lis.illinois.edu/cgi-bin/ethnea/search.py?Fname=FIRSTNAME&Lname=LASTNAME&format=json"
    if d["Genni"][i] == True or d["Ethnea"][i] == True:
        if d["Genni"][i] == True:
            fname = d["First Name"][i]
            if (' ' in fname) == True:
                fname = fname.split()
                fname = "%20".join(fname)
        else:
            fname = ""
        if d["Ethnea"][i] == True:
            lname = d["Last Name"][i]
            if (' ' in lname) == True:
                lname = lname.split()
                lname = "%20".join(lname)
        else:
            lname = ""
        url = url.replace('FIRSTNAME',fname).replace('LASTNAME',lname)
#Perform querry
        data = ast.literal_eval(urlopen(url).read().rstrip().decode('UTF-8'))
        d.loc[i, "Ethnea"] = data['Ethnea']
        d.loc[i, "Genni"] = data['Genni']

### 1a. Ethnicity groupings

## 2. Salary annualization

In [6]:
import math

In [7]:
d["AdjSalary"] = 1
for i in range(len(d)):
    d.loc[i,"AdjSalary"] = d["Annual Salary"][i].replace('$',"").replace(',',"").replace('*',"")
    frq = 1
    if math.isnan(d["%"][i]) == False:
        frq = float(d["%"][i])
    if math.isnan(d["FTE"][i]) == False:
        frq = float(d["FTE"][i])
    if math.isnan(d["PAY_FRCTN"][i]) == False:
        frq = float(d["PAY_FRCTN"][i])
    if frq > 1:
        d.loc[i,"AdjSalary"] = float(d.loc[i,"AdjSalary"]) / (frq/100)
    if frq <= 1:
        d.loc[i,"AdjSalary"] = float(d.loc[i,"AdjSalary"]) / frq

## 3. Total number of postdocs


In [8]:
d["PostdocNum"] = 0.1
univ = list(set(d["Institution"]))
for i in range(len(univ)):
    num = sum([1 for j in list(d["Institution"]) if j==univ[i]])
    ind = d.loc[d["Institution"] == univ[i]].index
    d.loc[ind, "PostdocNum"] = num

## 4. Postdoc titles

In [88]:
import re

In [17]:
#Turn 91 different titles into a handfull of defining words
words = {'Fellow':["fellow"],
        'Associate': ["asso", "aso"],
        'Researcher' : ["res","not_resident"],
         'Scholar' : ["scholar"],
         'Intern' : ["intern"],
         'Trainee' : ["trainee"],
         'Teaching' : ["teach"],
         'Senior' : ["senior", "sr"],
         'Assistant' : ["asst"],
         'Clinical' : ["clinical"], 
        'Assistant' : ["asst", "assistant"],
        'Faculty' : ["fac"]}

for word in words.keys():
    d[word] = 0

In [18]:
for i in range(len(d)):
    target = d.loc[i,"Title"].lower()
    for word in words.keys():
        search_result = 0
        if len(words[word]) == 1:
            search_result = len(re.findall(words[word][0], target))
        if len(words[word]) > 1:
            if "not_" in words[word][1]:
                word1 = re.sub("not_","",words[word][1])
                search_result = len(re.findall(words[word][0], target)) - len(re.findall(word1, target))
            else:
                search_result = len(re.findall(words[word][0], target)) + len(re.findall(words[word][1], target))
        if search_result>0:
            d.loc[i, word] = 1

In [19]:
d.head()

,Institution,Title,Annual Salary,APPT FTR BASIS,APPT FRACTION,AMOUNT OF SALARY PAID FROM GENERAL FUND,%,FTE,Time,PAY_FRCTN,...,Associate,Scholar,Trainee,Clinical,Assistant,Researcher,Intern,Professor,Faculty,Senior
0,ALBANY,Postdoctoral Associate,"$35,000.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0
1,ALBANY,Postdoctoral Associate,"$35,901.32",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0
2,ALBANY,Postdoctoral Associate,"$36,000.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0
3,ALBANY,Postdoctoral Associate,"$36,000.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0
4,ALBANY,Postdoctoral Associate,"$36,000.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0


## 5. Postdoc location

In [3]:
univ_info = pd.read_csv("/Users/NikiAthanasiadou/FoR/Info-Data/Salaries/Univ_info.csv") 

In [6]:
d["Metro_code"] = 0
d["pct_urban"] = 0
d["State"] = ""
for i in range(len(d)):
    for j in range(len(univ_info)):
        if univ_info.loc[j, "University_FoiA"].strip() == d.loc[i, "Institution"].strip():
            d.loc[i, "Metro_code"] = univ_info.loc[j, "metro code_13"]
            d.loc[i, "pct_urban"] = univ_info.loc[j, "pct_urban_10"]
            d.loc[i, "State"] = univ_info.iloc[j, 3]

7906

## 6. Postdoc-to-student ratio


In [25]:
d["no_students"] = 0
for i in range(len(d)):
    for j in range(len(univ_info)):
        if univ_info.loc[j, "University_FoiA"].strip() == d.loc[i, "Institution"].strip():
            d.loc[i, "no_students"] = univ_info.loc[j, "no_students_17"]
d["postdocs_per_1000stdduents"] = d["PostdocNum"]/(d["no_students"]/1000)

## 7. NIH grants

In [56]:
univ_info = pd.read_csv("/Users/NikiAthanasiadou/FoR/Info-Data/Salaries/Univ_info.csv") 
univ_info.head()

,University_FoiA,ZIP,County,Unnamed: 3,metro_13,metro code_13,pct_urban_10,no_students_17,2017_NIH,NIH_top
0,UM_ANN-ARBOR,48109.0,Washtenaw,MI,Y,2.0,83.6,43651.0,509085057.0,1.0
1,University of North Carolina Chapel Hill,27599.0,Orange,NC,Y,2.0,71.5,29084.0,410444710.0,2.0
2,University of Colorado Denver,80204.0,Denver,CO,Y,1.0,100.0,23671.0,215712348.0,3.0
3,University of Texas Southwestern,75390.0,Dallas,TX,Y,1.0,99.3,0.0,177844392.0,4.0
4,The Ohio State University-Main Campus,43210.0,Franklin,OH,Y,1.0,98.6,58663.0,169695210.0,5.0


In [68]:
d["NIH_grants"] = ""
d["NIH_order"] = 0
for i in range(len(d)):
    for j in range(len(univ_info)):
        if univ_info.loc[j, "University_FoiA"].strip() == d.loc[i, "Institution"].strip():
            d.loc[i, "NIH_grants"] = univ_info.loc[j, "2017_NIH"]
            d.loc[i, "NIH_order"] = univ_info.loc[j,"NIH_top"]

## 8. Ethicity words

In [97]:
a = list(set(d["Ethnea"]))
a.pop(a.index('False'))
a.pop(a.index('UNKNOWN'))
a.pop(a.index('TOOSHORT'))
a = [word.split("-") for word in a]
a = list(set([val for sublist in a for val in sublist]))
print(a)

['ROMANIAN', 'ENGLISH', 'BALTIC', 'HISPANIC', 'DUTCH', 'GERMAN', 'NORDIC', 'INDIAN', 'VIETNAMESE', 'HUNGARIAN', 'JAPANESE', 'ISRAELI', 'SLAV', 'CHINESE', 'TURKISH', 'THAI', 'MONGOLIAN', 'AFRICAN', 'GREEK', 'FRENCH', 'KOREAN', 'ITALIAN', 'CARIBBEAN', 'INDONESIAN', 'ARAB']


In [98]:
for word in a:
    d[word] = 0

In [99]:
for i in range(len(d)):
    target = d.loc[i,"Ethnea"]
    for word in a:
        search_result = 0
        search_result = len(re.findall(word, target))
        if search_result>0:
            d.loc[i, word] = 1

In [102]:
d.columns

Index(['Unnamed: 0', 'Last Name', 'First Name', 'Ethnea', 'Genni',
       'Annual Salary', 'AdjSalary', 'Title', 'Teaching', 'Fellow',
       'Associate', 'Scholar', 'Trainee', 'Clinical', 'Assistant',
       'Researcher', 'Intern', 'Professor', 'Faculty', 'Senior', 'Institution',
       'Department', 'State', 'Metro_code', 'pct_urban', 'PostdocNum',
       'no_students', 'postdocs_per_1000stdduents', 'NIH_grants', 'NIH_order',
       'ROMANIAN', 'ENGLISH', 'BALTIC', 'HISPANIC', 'DUTCH', 'GERMAN',
       'NORDIC', 'INDIAN', 'VIETNAMESE', 'HUNGARIAN', 'JAPANESE', 'ISRAELI',
       'SLAV', 'CHINESE', 'TURKISH', 'THAI', 'MONGOLIAN', 'AFRICAN', 'GREEK',
       'FRENCH', 'KOREAN', 'ITALIAN', 'CARIBBEAN', 'INDONESIAN', 'ARAB'],
      dtype='object')

### (Preserve backup)

In [71]:
cp = d.copy()

### (Save to disk)

In [7]:
out = d[['Last Name', 'First Name', 'Ethnea', 'Genni', 'Annual Salary', 'AdjSalary', 
        'Title','Teaching', 'Fellow', 'Associate',
       'Scholar', 'Trainee', 'Clinical', 'Assistant', 'Researcher',
       'Intern', 'Professor', 'Faculty', 'Senior', 'Institution',  'Department', 'State', 'Metro_code',
       'pct_urban', 'PostdocNum','no_students', 'postdocs_per_1000stdduents',
       'NIH_grants', 'NIH_order','ROMANIAN', 'ENGLISH', 'BALTIC', 'HISPANIC', 'DUTCH', 'GERMAN',
       'NORDIC', 'INDIAN', 'VIETNAMESE', 'HUNGARIAN', 'JAPANESE', 'ISRAELI',
       'SLAV', 'CHINESE', 'TURKISH', 'THAI', 'MONGOLIAN', 'AFRICAN', 'GREEK',
       'FRENCH', 'KOREAN', 'ITALIAN', 'CARIBBEAN', 'INDONESIAN', 'ARAB']]
out.to_csv("Postdoc_salary_Working.csv")

### (Load version saved to disk)

In [1]:
import pandas as pd

In [2]:
d = pd.read_csv("/Users/NikiAthanasiadou/FoR/Info-Data/Salaries/Postdoc_salary_Working.csv")
d.head()

,Unnamed: 0,Last Name,First Name,Ethnea,Genni,Annual Salary,AdjSalary,Title,Teaching,Fellow,...,THAI,MONGOLIAN,AFRICAN,GREEK,FRENCH,KOREAN,ITALIAN,CARIBBEAN,INDONESIAN,ARAB
0,0,NaN,NaN,False,False,"$35,000.00",35000.00,Postdoctoral Associate,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,NaN,NaN,False,False,"$35,901.32",35901.32,Postdoctoral Associate,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,NaN,NaN,False,False,"$36,000.00",36000.00,Postdoctoral Associate,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,NaN,NaN,False,False,"$36,000.00",36000.00,Postdoctoral Associate,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,NaN,NaN,False,False,"$36,000.00",36000.00,Postdoctoral Associate,0,0,...,0,0,0,0,0,0,0,0,0,0
